# File to experiment with different machine learning models.

In [1]:
from lib.DataObject import DataObject
import lib.DataObjectUtils as util
import torch
import pickle
import torch.nn as nn
from lib.DataHandler import DataAcquisitionHandler

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
# load data from data/* using pickle
filename = 'data/handler_box_data_full_Oct_30_2023.pkl'
with open(filename, 'rb') as f:
    handler = pickle.load(f)

In [ ]:
# Data filter visitor test

data = DataObject(handler.get_data())

data.accept(util.BandpassFilterVisitor(low=0.1, high=10))
data.accept(util.BandstopFilterVisitor(low=49, high=51))

formatted_data = data.get_data(decorator=util.MakeTensorWindowsDataDecorator())

print(formatted_data[0].shape)
print(len(formatted_data[1]))
print(len(formatted_data[1][0]))
print(len(formatted_data[1][0][0]))
print(len(formatted_data[1][0][0][0]))
print(len(formatted_data[1][2][0][0]))
print(formatted_data)

In [1]:
# create p300Model
class EEGNet(torch.nn.Module):
    """
    Pytorch implementation of EEGNet
    """

    def __init__(self):
        super(EEGNet, self).__init__()

        # block 1 nn.sequential
        self.block1 = torch.nn.Sequential(
            # Conv2D
            nn.Conv2d(1, 16, (1, 51), padding=0, bias=False),
            
            # Batch norm
            nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # DepthwiseConv2D
            nn.Conv2d(16, 32, (2, 1), groups=16, bias=False),

            # Batch norm
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # ELU Activation
            nn.ELU(alpha=1.0),

            # Avg Pooling 2D
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0),

            # Spacial Dropout 2D
            nn.Dropout2d(p=0.15)
        )

        # block 2 nn.sequential
        self.block2 = torch.nn.Sequential(
            # Separable Conv2D
            nn.Conv2d(32, 32, (1, 15), padding=0, bias=False),

            # Batch norm
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # ELU Activation
            nn.ELU(alpha=1.0),

            # Avg Pooling 2D
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),

            # Spacial Dropout 2D
            nn.Dropout2d(p=0.15)
        )

        # Fully Connected Layer
        self.fc = nn.Linear(in_features=736, out_features=2, bias=True)

    def forward(self, x):
        # block 1
        x = self.block1(x)

        # block 2
        x = self.block2(x)

        # flatten
        x = x.view(x.shape[0], -1)

        # fc
        x = self.fc(x)

        return x

NameError: name 'torch' is not defined